In [5]:
import altair as alt
import pandas as pd
df = pd.read_csv("base de datos turismo - copia.csv", sep=";")
df = df[~df["Nacionalidad"].str.contains("TOTAL", case=False, na=False)]
df.head(5)

,Nacionalidad,Continente,ene-09,feb-09,mar-09,abr-09,may-09,jun-09,jul-09,ago-09,...,jul-24,ago-24,sept-24,oct-24,nov-24,dic-24,ene-25,feb-25,mar-25,abr-25
0,ANGOLA,AFRICA,6.0,4.0,3.0,4.0,0.0,4.0,3.0,3.0,...,0.0,3.0,1.0,20.0,2.0,1.0,3.0,0.0,1.0,3.0
1,ARGELIA,AFRICA,1.0,4.0,22.0,18.0,4.0,36.0,50.0,2.0,...,1.0,2.0,0.0,3.0,0.0,3.0,6.0,1.0,1.0,2.0
2,CABO VERDE,AFRICA,0.0,3.0,0.0,1.0,8.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,EGIPTO,AFRICA,12.0,8.0,3.0,0.0,11.0,3.0,2.0,5.0,...,7.0,8.0,7.0,9.0,10.0,14.0,10.0,7.0,6.0,9.0
4,GHANA,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,5.0,12.0,1.0,2.0,1.0,5.0,1.0,1.0,3.0,0.0


Acá apliqué lo visto en clases: importé las librerias, subi el archivo csv a colab, especifiqué que la separación era con puntos y coma, borre las casillas con los totales (en caso de que se me haya pasado alguna en la base de datos) y le di a df.head para visualizar la tabla preliminar.

In [9]:
df_largo = df.melt(
    id_vars=["Nacionalidad", "Continente"],
    var_name="Mes",
    value_name="Turistas")

Acá convertí la tabla a formato largo para

In [ ]:
df_largo["Turistas"] = df_largo["Turistas"].replace({"\.": "", ",": ""}, regex=True)
df_largo["Turistas"] = pd.to_numeric(df_largo["Turistas"], errors="coerce").fillna(0)

Aca

In [11]:
meses_es = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr", "may": "May",
    "jun": "Jun", "jul": "Jul", "ago": "Aug", "sept": "Sep", "oct": "Oct",
    "nov": "Nov", "dic": "Dec"
}
for esp, eng in meses_es.items():
    df_largo["Mes"] = df_largo["Mes"].str.replace(f"^{esp}", eng, regex=True)
    df_largo["Fecha"] = pd.to_datetime(df_largo["Mes"], format="%b-%y", errors="coerce")
df_largo = df_largo.dropna(subset=["Fecha"])
df_largo["Mes_Año"] = df_largo["Fecha"].dt.to_period("M").astype(str)
meses_es_inv = {
    1: "ene", 2: "feb", 3: "mar", 4: "abr", 5: "may", 6: "jun",
    7: "jul", 8: "ago", 9: "sept", 10: "oct", 11: "nov", 12: "dic"
}
df_largo["Etiqueta_esp"] = df_largo["Fecha"].dt.month.map(meses_es_inv) + " " + df_largo["Fecha"].dt.year.astype(str)

al darme cuenta que pandas no reconocia los meses del dataframe, procedí a "traducirlos" para que pudiera interpretar los meses. También cree una columna simplificada con el mes y año (como originalmente aparecen en mi base).

In [13]:
paises_objetivo = ["ARGENTINA", "BRASIL", "PERU", "BOLIVIA", "ESTADOS UNIDOS", "COLOMBIA"]
df_filtrado = df_largo[df_largo["Nacionalidad"].isin(paises_objetivo)]


Aqui seleccione los países de origen con mayor flujo de turistas hacia Chile. No pusé mas para no saturar de información a la visualización.

In [14]:
meses_label = df_filtrado[df_filtrado["Fecha"].dt.month.isin([1,4,7,10])].drop_duplicates(subset=["Mes_Año"])

Aquí decidí solo mostrar intervalos de a tres meses en el eje x para no saturarlo, debido a que cuando lo visualicé posteriormente este era inenteligible por el exceso de caracteres.

In [16]:


label_map_js = {...}
label_map_js = {row["Mes_Año"]: row["Etiqueta_esp"] for _, row in meses_label.iterrows()}

label_expr = "{" + ", ".join([f'"{k}": "{v}"' for k,v in label_map_js.items()]) + "}[datum.value] || ''"

En este paso, quise mostrar en el grafico los nombres de los meses en español (como "ene 2015" o "jul 2022") en lugar del formato que aparece por defecto. Creé un diccionario que asocia cada valor de mes con su etiqueta en español. Despues arme una expresión de javascript que Altair puede interpretar para personalizar las etiquetas del eje X.

In [18]:
chart = alt.Chart(df_filtrado).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X(
        "Mes_Año:O",
        title="Mes",
        axis=alt.Axis(
            labelAngle=-45,
            labelFontSize=8,
            labelExpr=label_expr
        )
    ),
    y=alt.Y("sum(Turistas):Q", title="Total de turistas"),
    color=alt.Color("Nacionalidad:N", title="País"),
    tooltip=["Mes_Año", "Nacionalidad", "sum(Turistas)"]
).properties(
    width=1000,
    height=400,
    title="Ingreso de turistas por país y mes (2009-2025)"
).interactive()
chart

alt.Chart(...)

finalmente, procedí a codificar el gráfico de barras con altair. Desarrollé incialmente un grafico de lineas, pero como encontré muy engorroso visualizar la información tanto de los países como  posibles tendencias, preferí utlizar este, teniendo cuidado con todos los detalles, desde que el titulo fuera lo suficientemente descriptivo como hasta que se visualizaran bien los meses en el eje x.